# Solving A transverse field Ising model

In [ ]:
using Yao
using Statistics: mean
using LinearAlgebra
using Plots

## Model Hamiltonians

### Transverse field Ising Model

$H = \sum\limits_{i=1}^{N-1} s_i^z s^z_{i+1} + h\sum\limits^{N}_{i=1} s_i^x$

where $s^\alpha = \frac 1 2 \sigma^\alpha$ is the spin operator.

In [ ]:
"""
for simplicity, we require an AbstractModel contains `size` and `periodic` members.
"""
abstract type AbstractModel{D} end

nspin(model::AbstractModel) = prod(model.size)

In [ ]:
"""
transverse field ising model, `h` is the strength of transverse field.
"""
struct TFI{D} <:AbstractModel{1}
    size::NTuple{D, Int}
    h::Float64
    periodic::Bool
    TFI(size::Int...; h::Real, periodic::Bool) = new{length(size)}(size, Float64(h), periodic)
end

In [ ]:
"""
get the bonds of a chain model.
"""
function get_bonds(model::AbstractModel{1})
    nbit, = model.size
    [(i, i%nbit+1) for i in 1:(model.periodic ? nbit : nbit-1)]
end

In [ ]:
"""
Get the hamiltonian of a TFI model
"""
function hamiltonian(model::TFI)
    nbit = nspin(model)
    sum(repeat(nbit, Z, (i,j)) for (i,j) in get_bonds(model))*0.25 +
    sum(put(nbit, i=>X) for i=1:nbit)*0.5model.h
end

In [ ]:
tfi_model = TFI(4; h=0.5, periodic=false)

In [ ]:
tfi_h = hamiltonian(tfi_model)

# A quantum circuit as an ansatz

Construct the following circuit as our ansatz

<div style="display:inline-block"><img style="float:left", src="images/fourqubit.png" width="500"></div>

In [ ]:
dump(Measure)

In [ ]:
Rxz() = chain(Rx(0.0), Rz(0.0))
MeasureOp(nbit::Int, i::Int) = Measure{nbit, 1, AbstractBlock}(Z, (i,), nothing, false)

function ansatz_circuit(nbit::Int, d::Int)
    circuit = chain(nbit)

    for i=1:nbit-1
        unit = chain(nbit)
        for j=1:d
            push!(unit, put(nbit, i=>Rxz()))
            push!(unit, put(nbit, nbit=>Rxz()))
            push!(unit, control(nbit, 1, 2=>shift(0.0)))
        end
        push!(circuit, unit)
    end
    push!(circuit, chain([MeasureOp(nbit, i) for i=1:nbit]))
    circuit
end

In [ ]:
example_circuit = ansatz_circuit(4, 1)

In [ ]:
nparameters(example_circuit)

In [ ]:
"""
    gensample(circuit, operator; nbatch=1024) -> Vector of Measure

Generate samples from MPS-inspired circuit. Here, `nbatch` means nshot.
`operator` is the operator to measure.
This function returns a vector of `Measure` gates, results are stored in `m.results`.
"""
function gensample(circuit, operator; nbatch=1024)
    mblocks = collect_blocks(Measure, circuit)
    for m in mblocks
        m.operator = operator
    end
    reg = zero_state(nqubits(circuit); nbatch=nbatch)
    reg |> circuit
    mblocks
end

In [ ]:
res = gensample(example_circuit, X; nbatch=1024)

In [ ]:
res[1].results

Get the energy through sampling

In [ ]:
"""
    VQE{MT, BT<:AbstractBlock}

Variational quantum eigensolver setup
* `MT` is the type of hamiltonian model,
* `BT` is the type of an input circuit.
"""
struct VQE{MT, BT<:AbstractBlock}
    model::MT
    circuit::BT
    nshots::Int
end

VQE(model, circuit::AbstractBlock; nshots::Int=1024) = VQE(model, circuit, nshots)

## Loss Function

In [ ]:
"""
Obtain the energy of a VQE application.
"""
function energy(vqe::VQE{<:TFI})
    # measuring Z
    eng = ising_energy(vqe.circuit, get_bonds(vqe.model), Z; nshots=vqe.nshots)
    # measuring transverse field term on basis X
    mblocks = gensample(vqe.circuit, X; nbatch=vqe.nshots)
    engx = sum(mean.([m.results for m in mblocks]))
    eng + vqe.model.h*engx/2
end

function ising_energy(circuit, bonds, basis; nshots::Int)
    mblocks = gensample(circuit, basis; nbatch=nshots)
    nspin = length(mblocks)
    local eng = 0.0
    for (a, b) in bonds
        eng += mean(mblocks[a].results .* mblocks[b].results)
    end
    eng/=4
end

In [ ]:
# obtain the energy through sampling
vqe = VQE(tfi_model, example_circuit; nshots=1024)
dispatch!(vqe.circuit, :random)
energy(vqe)

In [ ]:
wave_function(circuit) = zero_state(nqubits(circuit)) |> circuit[1:end-1]

In [ ]:
# obtain the exact <H> through wave function
energy_exact(vqe) = expect(hamiltonian(vqe.model), wave_function(vqe.circuit)) |> real
energy_exact(vqe)

## Training

Sequential optimization
1. obtain the gradient of one parameter a time $\frac{\partial E_\theta}{\partial \theta} = \frac 1 2 (E_{\theta+\pi/2}-E_{\theta-\pi/2})$
2. perform gradient descent update of this parameter $\theta\rightarrow \theta-\alpha \frac{\partial E_\theta}{\partial \theta}$
3. sweep all parameters until convergence.

In [ ]:
fidelity(psi, VG) = abs(psi' * VG)

In [ ]:
nparameters(vqe.circuit)

In [ ]:
"""
obtain the gradient of a parameter in a rotation gate/cphase gate.
"""
function opgrad(vqe::VQE, r)
    dispatch!(+, r, π/2)
    E₊ = energy(vqe)
    dispatch!(-, r, π)
    E₋ = energy(vqe)
    dispatch!(+, r, π/2)
    0.5*(E₊ - E₋)
end

In [ ]:
using Flux: ADAM, Optimise

In [ ]:
function train!(vqe::VQE; maxiter::Int=200, α::Real=0.3)
    circuit, model = vqe.circuit, vqe.model
    rots = collect_blocks(Union{RotationGate, ControlBlock{<:Any, <:ShiftGate}}, circuit)
    loss_history = Float64[]
    params = vcat(parameters.(rots)...)
    optimizer = ADAM(α)
    for i in 0:maxiter
        grad = opgrad.(Ref(vqe), rots)
        Optimise.update!(optimizer, params, grad)
        dispatch!.(rots, params)
        push!(loss_history, energy_exact(vqe)/nspin(model))
        
        i%10 == 0 && println("Iter $i, E/N = $(loss_history[end])")
    end
    loss_history, circuit
end

In [ ]:
lattice_size = 4
vqe = VQE(TFI(lattice_size; h=0.5, periodic=false),
    ansatz_circuit(lattice_size, 2);
    nshots=1024);

In [ ]:
# obtain the exact ground state energy
res = eigen(mat(hamiltonian(vqe.model)) |> Matrix)
EG = res.values[1]/nspin(vqe.model)
@show EG
VG = res.vectors[:,1];

In [ ]:
nparameters(vqe.circuit)

In [ ]:
dispatch!(vqe.circuit, :random)
loss_history, circuit = train!(vqe; maxiter=100, α=0.3);

In [ ]:
M = length(loss_history)
plot(0:M-1, [loss_history, fill(EG, M)], label=["QMPS", "Exact"], lw=3, ylabel="Energy")

## Task 1: The problem of vanishing gradient
Change the circuit size (as well as model lattice size), show the variance of gradient items.

#### Hint: useful functions
* using Statistics: var
* Use `energy_exact` instead of `energy` to avoid sampling error

## Task 2: Two qubits simulating a four-qubit systems

Hadamard Gate with dashed box is applied only when measuring on the X basis.
<div style="display:inline-block"><img style="float: left", src="images/hgate.png" width="50"/></div>

The following gates Measure and Reset a qubit to 0.

<div style="display:inline-block"><img style="float:left", src="images/mreset.png" width="100"></div>

The goal of this section is to build the MPS-inspired sampler as our ansatz

<div style="display:inline-block"><img style="float:left", src="images/twoqubit.png"></div>

### Reference: Variational Quantum Eigensolver with Fewer Qubits

*Jin-Guo Liu, Yi-Hong Zhang, Yuan Wan, Lei Wang*

https://arxiv.org/abs/1902.02663

In [ ]:
# Hint
MeasureAndReset(nbit::Int, i::Int) = Measure{nbit, 1, AbstractBlock}(Z, (i,), 0, false)

In [ ]:
"""
    ansatz_circuit_2qubit(nbit::Int, d::Int)

Construct the above ansatz, `nrepeat` is the number of measure operations,
`d` is the depth of each block between two measurements.
"""
function ansatz_circuit_2qubit(nbit::Int, d::Int)
    # write your code here
end

In [ ]:
lattice_size = 4
vqe = VQE(TFI(lattice_size; h=0.5, periodic=false),
    ansatz_circuit_2qubit(lattice_size, 2);
    nshots=1024);

In [ ]:
nparameters(vqe.circuit)

In [ ]:
dispatch!(vqe.circuit, :random)
loss_history, circuit = train!(vqe; maxiter=100, α=0.3);

In [ ]:
M = length(loss_history)
plot(0:M-1, [loss_history, fill(EG, M)], label=["QMPS", "Exact"], lw=3, ylabel="Energy")